# Jocelin Liteanu - Observing with `astroplan`

# Observing Window:
 * Start: `Jul 27, 2018 12:00:00 UTC`
 * End: `Jul 28, 2018 12:00:00 UTC`

---

# Locations

- Mawson Peak, Heard Island
- Porto De Manaus, Bazil
- Inis na Bró, County Kerry, Ireland

---

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import astropy.units as u
from astropy.time import Time
from astropy.coordinates import SkyCoord
import pytz

from astroplan.plots import plot_sky, plot_airmass
from astroplan import Observer, FixedTarget, time_grid_from_range, observability_table, moon_illumination
from astroplan import AirmassConstraint, MoonSeparationConstraint

# Part I

## Set up the locations

In [2]:
mawson_peak = Observer(longitude = 73 * u.deg + 30 * u.arcmin + 59.99 * u.arcsec,
                    latitude = -53 * u.deg + 5 * u.arcmin + 60 * u.arcsec,              
                    elevation = 9006 * u.m,
                    timezone = 'Antarctica/Mawson',
                    name = "mawson_tz"
                    )

mawson_peak

<Observer: name='mawson_tz',
    location (lon, lat, el)=(73.51666388888889 deg, -52.9 deg, 9006.000000000862 m),
    timezone=<DstTzInfo 'Antarctica/Mawson' -000:00:00 STD>>

In [3]:
porto = Observer(longitude =  -60* u.deg +  1* u.arcmin + .6 * u.arcsec,
                    latitude = -3 * u.deg + 8 * u.arcmin +  21* u.arcsec,
                    elevation = 92 * u.m,
                    timezone = 'America/Manaus',
                    name = "porto_tz"
                    )
porto

<Observer: name='porto_tz',
    location (lon, lat, el)=(-59.983166666666676 deg, -2.860833333333334 deg, 91.99999999951184 m),
    timezone=<DstTzInfo 'America/Manaus' LMT-1 day, 20:00:00 STD>>

In [4]:
inis = Observer(longitude = -10 * u.deg + 36 * u.arcmin + 32.87 * u.arcsec,
                    latitude = 52 * u.deg + 3 * u.arcmin + 28.30 * u.arcsec,
                    elevation = 175 * u.m,
                    timezone = 'Europe/Dublin',
                    name = "inis_tz"
                    )


## Determine the *Local Time* at beginning of observing window at each location.

In [5]:
current_time = Time.now()

In [6]:
mawson_peak

<Observer: name='mawson_tz',
    location (lon, lat, el)=(73.51666388888889 deg, -52.9 deg, 9006.000000000862 m),
    timezone=<DstTzInfo 'Antarctica/Mawson' -000:00:00 STD>>

In [7]:
mawson_peak.timezone

<DstTzInfo 'Antarctica/Mawson' -000:00:00 STD>

In [8]:
local_now = current_time.to_datetime(mawson_peak.timezone)
print(local_now)

2018-02-21 01:24:35.532834+05:00


In [9]:
porto.timezone

<DstTzInfo 'America/Manaus' LMT-1 day, 20:00:00 STD>

In [10]:
local_now = current_time.to_datetime(porto.timezone)
print(local_now)

2018-02-20 16:24:35.532834-04:00


In [11]:
inis.timezone

<DstTzInfo 'Europe/Dublin' LMT-1 day, 23:35:00 STD>

In [12]:
local_now = current_time.to_datetime(inis.timezone)
print(local_now)

2018-02-20 20:24:35.532834+00:00


## Determine the start/end time and length of the observing run at each location

* All times should be in UTC
* Length of run is from Astronomical Evening Twilight to Astronomical Morning Twilight
* If your length is > 24 hr or < 0 hr you pick the wrong sunset/sunrise

In [13]:
start_time = Time("2018-07-27 12:00:00")

In [14]:
end_time = Time("2018-07-28 12:00:00")

In [15]:
start_time

<Time object: scale='utc' format='iso' value=2018-07-27 12:00:00.000>

In [16]:
astro_set_m = mawson_peak.twilight_evening_astronomical(start_time, which='nearest')
astro_rise_m = mawson_peak.twilight_morning_astronomical(start_time, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_set_m))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_rise_m))


Astronomical Evening Twilight starts at 2018-07-27 13:31:34.383 UTC
Astronomical Morning Twilight starts at 2018-07-28 00:52:46.206 UTC


In [17]:
observing_length_mawson = (astro_rise_m - astro_set_m).to(u.h)
print("You can observe for {0:.1f} at Mawson/Antarctica tonight".format(observing_length_mawson))

You can observe for 11.4 h at Mawson/Antarctica tonight


In [18]:
astro_set_p = porto.twilight_evening_astronomical(start_time, which='nearest')
astro_rise_p = porto.twilight_morning_astronomical(start_time, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_rise_p))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_set_p))

Astronomical Evening Twilight starts at 2018-07-28 08:54:13.156 UTC
Astronomical Morning Twilight starts at 2018-07-27 23:18:41.731 UTC


In [19]:
observing_length_porto = (astro_rise_p - astro_set_p).to(u.h)
print("You can observe for {0:.1f} at Brazil/Manaus tonight".format(observing_length_porto))


You can observe for 9.6 h at Brazil/Manaus tonight


In [20]:
astro_set_i = inis.twilight_evening_astronomical(start_time, which='nearest')
astro_rise_i = inis.twilight_morning_astronomical(start_time, which='next')

print("Astronomical Evening Twilight starts at {0.iso} UTC".format(astro_rise_i))
print("Astronomical Morning Twilight starts at {0.iso} UTC".format(astro_set_i))

Astronomical Evening Twilight starts at 2018-07-28 01:36:28.016 UTC
Astronomical Morning Twilight starts at 2018-07-27 23:52:08.435 UTC


In [21]:
observing_length_inis = (astro_rise_i - astro_set_i).to(u.h)
print("You can observe for {0:.1f} at Europe/Dublin tonight".format(observing_length_inis))

You can observe for 1.7 h at Europe/Dublin tonight


# Part II

* I have given you three locations (above), some constraints, and list of targets: `./Data/TargetList.csv`
* Your job is to determine which of the three locations can see the most targets for the longest time, given the constraints.
* Points awarded will depend on how clearly you support your answer.

## Set up the targets (`./Data/TargetList.csv`)

In [22]:
target_table = pd.read_csv('./Data/TargetList.csv')

In [23]:
table_values = target_table.as_matrix()

In [24]:
table_values

array([['HCG_98', 23.903544, 0.373281],
       ['HCG_65', 13.498313000000001, -29.499610999999998],
       ['HCG_66', 13.64265, 57.304339],
       ['HCG_67', 13.817646, -7.205427],
       ['HCG_85', 18.839536, 73.349942],
       ['HCG_87', 20.803299, -19.840636],
       ['HCG_88', 20.873002, -5.757898],
       ['HCG_92', 22.600143, 33.965767]], dtype=object)

In [25]:
targets = [FixedTarget(coord=SkyCoord(ra = RA*u.hourangle, dec = DEC*u.deg), name=Name)
           for Name, RA, DEC in table_values]

In [26]:
targets

[<FixedTarget "HCG_98" at SkyCoord (ICRS): (ra, dec) in deg (358.55316, 0.373281)>,
 <FixedTarget "HCG_65" at SkyCoord (ICRS): (ra, dec) in deg (202.474695, -29.499611)>,
 <FixedTarget "HCG_66" at SkyCoord (ICRS): (ra, dec) in deg (204.63975, 57.304339)>,
 <FixedTarget "HCG_67" at SkyCoord (ICRS): (ra, dec) in deg (207.26469, -7.205427)>,
 <FixedTarget "HCG_85" at SkyCoord (ICRS): (ra, dec) in deg (282.59304, 73.349942)>,
 <FixedTarget "HCG_87" at SkyCoord (ICRS): (ra, dec) in deg (312.049485, -19.840636)>,
 <FixedTarget "HCG_88" at SkyCoord (ICRS): (ra, dec) in deg (313.09503, -5.757898)>,
 <FixedTarget "HCG_92" at SkyCoord (ICRS): (ra, dec) in deg (339.002145, 33.965767)>]

## Constraints

- Airmass < 2
- Moon Separation > 35 degrees

In [27]:
from astroplan import AltitudeConstraint, AirmassConstraint, AtNightConstraint
from astroplan import observability_table
from astroplan import MoonSeparationConstraint

In [28]:
constraints = [AirmassConstraint(2)]

In [29]:
constraints.append(MoonSeparationConstraint(35*u.deg))

### Location 1

In [30]:
observing_range_m = [astro_set_m, astro_rise_m]

In [31]:
observing_table_m = observability_table(constraints, mawson_peak, targets, time_range=observing_range_m)

print(observing_table_m)

target name ever observable always observable fraction of time observable
----------- --------------- ----------------- ---------------------------
     HCG_98            True             False           0.391304347826087
     HCG_65            True             False         0.34782608695652173
     HCG_66           False             False                         0.0
     HCG_67            True             False         0.21739130434782608
     HCG_85           False             False                         0.0
     HCG_87           False             False                         0.0
     HCG_88           False             False                         0.0
     HCG_92           False             False                         0.0


In [32]:
window_range_m = [start_time, end_time]
window_time_grid_m = time_grid_from_range(window_range_m)

In [33]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):

    if observing_table_m['ever observable'][i]:
        ax = plot_airmass(my_object, mawson_peak, window_time_grid_m, max_airmass=2.0)

ax.vlines(astro_set_m.datetime, 1,3, color='r', linewidth=5)
ax.vlines(astro_rise_m.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

In [34]:
for i, my_object in enumerate(targets):

    if observing_table_m['ever observable'][i]:
        name = observing_table_m['target name'][i]
        obj_frac = observing_table_m['fraction of time observable'][i]
        obj_time = obj_frac * observing_length_mawson
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe HCG_98 for 4.44 h
You can observe HCG_65 for 3.95 h
You can observe HCG_67 for 2.47 h


### Location 2

In [35]:
observing_range_p = [astro_set_p, astro_rise_p]

In [36]:
observing_table_p = observability_table(constraints, porto, targets, time_range=observing_range_p)

print(observing_table_p)

target name ever observable always observable fraction of time observable
----------- --------------- ----------------- ---------------------------
     HCG_98            True             False                        0.55
     HCG_65            True             False                         0.2
     HCG_66           False             False                         0.0
     HCG_67            True             False                        0.25
     HCG_85           False             False                         0.0
     HCG_87           False             False                         0.0
     HCG_88           False             False                         0.0
     HCG_92            True             False                         0.6


In [37]:
window_range_p = [start_time, end_time]
window_time_grid_p = time_grid_from_range(window_range_p)

In [38]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):

    if observing_table_p['ever observable'][i]:
        ax = plot_airmass(my_object, porto, window_time_grid_p, max_airmass=2.0)

ax.vlines(astro_set_p.datetime, 1,3, color='r', linewidth=5)
ax.vlines(astro_rise_p.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

In [39]:
for i, my_object in enumerate(targets):

    if observing_table_p['ever observable'][i]:
        name = observing_table_p['target name'][i]
        obj_frac = observing_table_p['fraction of time observable'][i]
        obj_time = obj_frac * observing_length_porto
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe HCG_98 for 5.28 h
You can observe HCG_65 for 1.92 h
You can observe HCG_67 for 2.40 h
You can observe HCG_92 for 5.76 h


### Location 3

In [40]:
observing_range_i = [astro_set_i, astro_rise_i]

In [41]:
observing_table_i = observability_table(constraints, inis, targets, time_range=observing_range_i)

print(observing_table_i)

target name ever observable always observable fraction of time observable
----------- --------------- ----------------- ---------------------------
     HCG_98           False             False                         0.0
     HCG_65           False             False                         0.0
     HCG_66            True              True                         1.0
     HCG_67           False             False                         0.0
     HCG_85            True              True                         1.0
     HCG_87           False             False                         0.0
     HCG_88           False             False                         0.0
     HCG_92            True              True                         1.0


In [42]:
window_range_i = [start_time, end_time]
window_time_grid_i = time_grid_from_range(window_range_i)

In [43]:
fig,ax = plt.subplots(1,1)
fig.set_size_inches(10,5)

fig.tight_layout()

for i, my_object in enumerate(targets):

    if observing_table_i['ever observable'][i]:
        ax = plot_airmass(my_object, inis, window_time_grid_i, max_airmass=2.0)

ax.vlines(astro_set_i.datetime, 1,3, color='r', linewidth=5)
ax.vlines(astro_rise_i.datetime, 1,3, color='r', linewidth=5)

ax.legend(loc=0,shadow=True);

In [44]:
for i, my_object in enumerate(targets):

    if observing_table_i['ever observable'][i]:
        name = observing_table_i['target name'][i]
        obj_frac = observing_table_i['fraction of time observable'][i]
        obj_time = obj_frac * observing_length_inis
        output = "You can observe {0:s} for {1:.2f}".format(name, obj_time.to(u.h))
        print(output)

You can observe HCG_66 for 1.74 h
You can observe HCG_85 for 1.74 h
You can observe HCG_92 for 1.74 h


In [45]:
#which location is the best to observe? 
#Location 1 is 3 objects in 10.6 hours
#Location 2 is 5 objects in 15.36 hours
#Location 3 is 3 objects in 5.22 hours
#The best to observe would be  most objects for the longest amount of time is porto Brazil.

### Due Mon Feb 19 - Noon
- `Make sure to change the filename to your name!`
- `Make sure to change the Title to your name!`